In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pokedex = pd.read_csv("../input/pokemon/pokemon.csv")
pokedex

In [ ]:
pokedex.drop([col for col in pokedex.columns if 'against' in col],axis=1,inplace=True)

pokedex.describe()

In [ ]:
pokedex.describe()

In [ ]:
import seaborn as sns

corr = pokedex.corr()
sns.heatmap(corr,
xticklabels=corr.columns.values,
yticklabels=corr.columns.values,cmap='OrRd', annot=False)

In [ ]:
corr

In [ ]:

sns.distplot(pokedex['generation'], bins=7, kde=False)

### from matplotlib import pyplot as plt
sns.set(style="whitegrid")
plt.figure(figsize=(8,6))
plt.title("Rozklad wagowy")
sns.histplot(data = pokedex[pokedex['generation']==34], x='experience_growth', kde=True)

In [ ]:
from matplotlib import pyplot as plt
sns.set(style="whitegrid") 
plt.figure(figsize=(8,6))
plt.title("Rozklad attaku gen 1") 
sns.histplot(data = pokedex[pokedex['generation']==1], x='experience_growth', kde=True)

In [ ]:
plt.figure(figsize=(8,6))
plt.ticklabel_format(style='plain',axis='y')
plt.title('atak od ilości męższczyzn przestrzeni generacji')
sns.scatterplot(data=pokedex, x='attack', y='percentage_male', hue='generation' )

In [ ]:
# pokedex.drop([col for col in pokedex.columns if 'against' in col],axis=1,inplace=True)
# pokedex.drop([col for col in pokedex.columns if 'base' in col],axis=1,inplace=True)
# pokedex.drop([ 'classfication', 'japanese_name', 'abilities', 'name', 'type1', 'type2', 'percentage_male', 'sp_attack', 'attack'], axis=1, inplace=True)

# pokedex[pokedex.columns]
pokedex.index[774]

In [ ]:
from sklearn.preprocessing import StandardScaler

pokedex.drop([ 'base_total'], axis=1, inplace=True)
pokedex.drop([col for col in pokedex.columns if 'against' in col],axis=1,inplace=True)
pokedex.drop([col for col in pokedex.columns if 'base' in col],axis=1,inplace=True)
pokedex.drop([ 'classfication', 'japanese_name', 'abilities', 'name', 'type1', 'type2'], axis=1, inplace=True)

pokedex.drop(pokedex.index[773], inplace=True)

p_data = pokedex
pokedex
scaler = StandardScaler()
pokedex[pokedex.columns] = scaler.fit_transform(pokedex)


In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(pokedex)
SimpleImputer()



pokedex = imp.transform(pokedex)

pokedex

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import calinski_harabasz_score

inertia_scores = []
silhouette_scores = []
calinski_harabaz_scores = []
# for i in range(2,11):
#     kmeans = KMeans(n_clusters=i).fit(pokedex)
#     inertia_scores.append(kmeans.inertia_)
#     silhouette_scores.append(metrics.silhouette_score(pokedex, kmeans.labels_))
#     calinski_harabaz_scores.append(metrics.calinski_harabaz_score(pokedex, kmeans.labels_))
# kmeans_model = KMeans(n_clusters=3, random_state=1).fit(X)

for i in range(2,11):
    kmeans = KMeans(n_clusters=i).fit(pokedex)
    inertia_scores.append(kmeans.inertia_)
    silhouette_scores.append(metrics.silhouette_score(pokedex, kmeans.labels_))
    #     calinski_harabaz_scores.append(metrics.calinski_harabaz_score(pokedex, kmeans.labels_))
    labels = kmeans.labels_
    calinski_harabaz_scores.append(calinski_harabasz_score(pokedex, labels))
#     metrics.calinski_harabasz_score(X, labels)

In [ ]:
plt.scatter(x=np.arange(2,11), y=inertia_scores)

In [ ]:
plt.scatter(x=np.arange(2,11), y=silhouette_scores)

In [ ]:
plt.scatter(x=np.arange(2,11), y=calinski_harabaz_scores)

In [ ]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(pokedex)

In [ ]:
centers = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_))
centers.columns = p_data.columns
centers

In [ ]:
pokedex_class = pd.DataFrame(scaler.inverse_transform(pokedex))
pokedex_class.columns = p_data.columns
pokedex_class['class'] = kmeans.labels_
pokedex_class.head(10)

In [ ]:
pokedex_class.groupby(pokedex_class['class']).size()

In [ ]:
from matplotlib import cm 
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import axes3d
fig = plt.figure(figsize=(10,10))
ax = Axes3D(fig)
ax.scatter(
pokedex_class['attack'],
pokedex_class['height_m'],
pokedex_class['percentage_male'],
cmap = cm.get_cmap("Spectral"),
c=kmeans.labels_
)
ax.set_xlabel('attack')
ax.set_ylabel('height_m')
ax.set_zlabel('percentage_male')
plt.ticklabel_format(style="plain",axis="x")
plt.show()


In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

plt.figure(figsize=(10,7))
dnd = dendrogram(linkage(pokedex, method="ward"))

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

plt.figure(figsize=(10,7))
dnd = dendrogram(linkage(pokedex, method="single"))

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

plt.figure(figsize=(10,7))
dnd = dendrogram(linkage(pokedex, method="average"))

In [ ]:
class_number = fcluster(linkage_matrix, t=5, criterion='maxclust')
centers_new = p_data.groupby(class_number).mean()
centers_new_df = pd.DataFrame(scaler.inverse_transform(centers_new))
centers_new_df.columns = p_data.columns
centers_new_df



In [ ]:
centers